In [26]:
import numpy as np
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, \
    AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, Softmax
from tensorflow.keras.models import Sequential
from keras.models import Model
from keras.initializers import glorot_uniform
from tensorflow import keras
from numpy import genfromtxt
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.keras.callbacks import TensorBoard
import os

from keras import backend as K
from tensorflow.keras.optimizers import Adam

In [27]:
# SNR
Training_Data = genfromtxt('Training_Data.csv', delimiter=",")
Training_Data = np.array(Training_Data)
#Training_Data = Training_Data[...,None]
print(Training_Data.shape)

Training_Labels = genfromtxt('Training_Labels.csv', delimiter=",")
Training_Labels = np.array(Training_Labels)
#Training_Labels = Training_Labels[...,None]
print(Training_Labels.shape)

Testing_Data = genfromtxt('Testing_Data.csv', delimiter=",")
Testing_Data = np.array(Testing_Data)
#Testing_Data = Testing_Data[...,None]
print(Testing_Data.shape)

Testing_Labels = genfromtxt('Testing_Labels.csv', delimiter=",")
Testing_Labels = np.array(Testing_Labels)
#Testing_Labels = Testing_Labels[...,None]
print(Testing_Labels.shape)


indices_1 = np.arange(Training_Data.shape[0])
np.random.shuffle(indices_1)
Training_Data = Training_Data[indices_1]
Training_Labels = Training_Labels[indices_1]

indices_2 = np.arange(Testing_Data.shape[0])
np.random.shuffle(indices_2)
Testing_Data = Testing_Data[indices_2]
Testing_Labels = Testing_Labels[indices_2]

(500000, 120)
(500000, 90)
(16800, 120)
(16800, 90)


In [28]:
metrics = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'),
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.F1Score(name='f1_score'),

]

In [29]:
# Define the model architecture
model = Sequential([
    Dense(480, activation='relu', input_shape=(120,)),
    BatchNormalization(),
    Dropout(0.2),
    Dense(240, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(120, activation='relu'),
    BatchNormalization(),
    Dense(90, activation='sigmoid')  # Sigmoid activation for multi-label output
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy',metrics=metrics)


In [30]:
batch = 500
epoch = 20

history = model.fit(Training_Data, Training_Labels, validation_split=0.01, epochs=epoch, batch_size=batch,
                      shuffle=True, verbose=1)
preds = model.evaluate(Testing_Data, Testing_Labels)


Epoch 1/20
990/990 [==============================] - 6s 4ms/step - loss: 0.1381 - tp: 3025147.0000 - fp: 1357049.0000 - tn: 38737936.0000 - fn: 1429853.0000 - accuracy: 0.9374 - precision: 0.6903 - recall: 0.6790 - auc: 0.9683 - f1_score: 0.0399 - val_loss: 0.0133 - val_tp: 42832.0000 - val_fp: 79.0000 - val_tn: 404921.0000 - val_fn: 2168.0000 - val_accuracy: 0.9950 - val_precision: 0.9982 - val_recall: 0.9518 - val_auc: 0.9998 - val_f1_score: 0.0433
Epoch 2/20
990/990 [==============================] - 4s 4ms/step - loss: 0.0101 - tp: 4353522.0000 - fp: 38269.0000 - tn: 40056728.0000 - fn: 101478.0000 - accuracy: 0.9969 - precision: 0.9913 - recall: 0.9772 - auc: 0.9995 - f1_score: 0.0439 - val_loss: 0.0019 - val_tp: 44762.0000 - val_fp: 10.0000 - val_tn: 404990.0000 - val_fn: 238.0000 - val_accuracy: 0.9994 - val_precision: 0.9998 - val_recall: 0.9947 - val_auc: 1.0000 - val_f1_score: 0.0426
Epoch 3/20
990/990 [==============================] - 4s 4ms/step - loss: 0.0050 - tp: 44066

In [31]:
print("loss = " + str(preds[0]))
print("TP = " + str(preds[1]))
print("FP = " + str(preds[2]))
print("TN = " + str(preds[3]))
print("FN = " + str(preds[4]))
print("BinaryAccuracy = " + str(preds[5]))
print("Precision = " + str(preds[6]))
print("Recall = " + str(preds[7]))
print("AUC = " + str(preds[8]))
#print("f1_score = " + str(preds[9]))


model.summary()

loss = 0.001843539415858686
TP = 150560.0
FP = 12.0
TN = 1360788.0
FN = 640.0
BinaryAccuracy = 0.99956876039505
Precision = 0.9999203085899353
Recall = 0.9957671761512756
AUC = 0.9994267225265503
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 480)               58080     
                                                                 
 batch_normalization_12 (Ba  (None, 480)               1920      
 tchNormalization)                                               
                                                                 
 dropout_8 (Dropout)         (None, 480)               0         
                                                                 
 dense_17 (Dense)            (None, 240)               115440    
                                                                 
 batch_normalization_13 (Ba  (None, 240)               9